In [ ]:
import os
import numpy as np

from sp_inference import logging, processes, model

In [ ]:
logSettings = {
    "output_directory":      "03_example_transient",
    "verbose":               True,
    "print_interval":        10
}

dataSettings = {
    "process_type":          "Dr31Di20Process",
    "drift_parameters":      [2, 3],
    "diffusion_parameters":  [1, 2],
    "standard_deviation":    0.01,
    "rng_seed":              0,
    "num_domain_points":     50,
    "num_time_points":       10,
    "domain_bounds":         [-1.5, 1.5],
    "time_bounds":           [0.1, 1]
}

modelSettings = {
    "params_to_infer":       "all",
    "model_type":            "fokker_planck",
    "is_stationary":         False
}

priorSettings = {
    "mean_function":         [lambda x: -x, lambda x: 3*np.ones(x.shape)],
    "gamma":                 0.5,
    "delta":                 0.5,
    "robin_bc":              False
}

feSettings = {
    "num_mesh_points":       500,
    "boundary_locations":    [-3.5, 3.5],
    "boundary_values":       [0, 0],
    "element_degrees":       [1, 1]
}

transientSettings = {
    "start_time":            0.1,
    "end_time":              1.1,
    "time_step_size":        0.025,
    "initial_condition":     lambda x: 1/(0.5*np.sqrt(2*np.pi)) * \
                                       np.exp(-0.5*np.square(x)/0.5**2)
}

solverSettings = {
    "rel_tolerance":         1e-6,
    "abs_tolerance":         1e-12,
    "max_iter":              10,
    "GN_iter":               5,
    "c_armijo":              1e-4,
    "max_backtracking_iter": 10
}

hessianSettings = {
    "num_eigvals":           35,
    "num_oversampling":      10
}

visualizationSettings = {
    "show":                  True,
    "time_points":           [0.1, 1.0]
}

In [ ]:
try:
    os.system('rm -r ' + logSettings["output_directory"])
except:
    pass

logger = logging.Logger(logSettings["verbose"],
                        logSettings["output_directory"],
                        logSettings["print_interval"])

In [ ]:
randGenerator = np.random.default_rng(dataSettings["rng_seed"])
randLocs = randGenerator.uniform(*dataSettings["domain_bounds"], dataSettings["num_domain_points"])
randTimes = np.linspace(*dataSettings["time_bounds"], dataSettings["num_time_points"])

dataSettings["domain_points"] = randLocs
dataSettings["time_points"] = randTimes
dataSettings["fem_settings"] = feSettings
dataSettings["solver_settings"] = transientSettings

processType = processes.get_process(dataSettings["process_type"])
process = processType(dataSettings["drift_parameters"],
                      dataSettings["diffusion_parameters"],
                      logger)

forwardNoisy, forwardExact = process.generate_data(modelSettings["model_type"],
                                                   modelSettings["is_stationary"],
                                                   dataSettings)

exactDrift = process.compute_drift(randLocs)
exactDiffusion = process.compute_squared_diffusion(randLocs)
exactParamValues = np.column_stack((exactDrift, exactDiffusion))

exactParamData = [randLocs, exactParamValues]
randForwardData = [randLocs, randTimes, forwardNoisy]
exactForwardData = [randLocs, randTimes, forwardExact]

misFitSettings = {
    "data_locations": randLocs,
    "data_times": randTimes,
    "data_values": forwardNoisy,
    "data_var": dataSettings["standard_deviation"]**2
}

In [ ]:
inferenceModel = model.SDEInferenceModel(modelSettings,
                                         priorSettings,
                                         feSettings,
                                         misFitSettings,
                                         transientSettings,
                                         logger)

priorMeanData, priorVarianceData, priorForwardData = inferenceModel.get_prior_info("Randomized")

In [8]:
test_data = priorForwardData[2]

In [ ]:
mapMeanData, mapVarianceData, mapForwardData, hessEigVals \
    = inferenceModel.compute_gr_posterior(solverSettings, hessianSettings)